# EarthArXiv Harvester
1. Run reference resolver to match bibcodes newly created since last week's harvest, and update the existing data file
2. Query the Crossref API to harvest all data by EarthArXiv DOI prefix
4. Compare API-harvested data with existing excel file; check for new records to be ingested, and metadata changes on records that already have a bibcode
5. Add new records and insert the metadata changes to the existing data file
6. Run the pyingest serializer to generate tagged format files: one for newly harvested records, one for metadata updates
7. Sftp the tag files into the adsx EarthArXiv directory

In [ ]:
from habanero import Crossref
import json
from datetime import datetime
from pyingest.serializers.classic import Tagged
import pandas as pd
import ast
import requests
import datetime
import re
import numpy as np

# Set the master_file for data harvest
excel_file = "eartharxiv_data.xlsx"
master_file = pd.read_excel(excel_file)
master_data = pd.DataFrame(master_file)

# Get today's date in the desired format
today_date = datetime.datetime.now().strftime("%y%m%d")

# Use the today_date variable to name your files
harvest_tagged_output = f"{today_date}_eaarx_harvest.tag"
metadata_tagged_output = f"{today_date}_eaarx_updates.tag"

## Resolve for newly added bibcodes

In [ ]:
# Read the Excel file into a DataFrame

# Get rows with a bibcode value
rows_with_bibcode = master_data[master_data['preprint_bib'] != '...................']
count_with_bibcode = len(rows_with_bibcode)
print(f"Rows with a bibcode: {count_with_bibcode}")

# Get rows with no bibcode
rows_with_no_bibcode = master_data[master_data['preprint_bib'] == '...................']
count_no_bibcode = len(rows_with_no_bibcode)
print(f"Rows with no bibcode: {count_no_bibcode}")

In [ ]:
# Filter the DataFrame based on the 'preprint_bib' column condition
filtered_df = master_data[master_data['preprint_bib'] == '...................']

# Isolate the DOIs and drop all the papers that have no DOIs (drop null values)
dois = filtered_df['preprint_doi'].dropna()

# Convert it from a DataFrame to a list
doi_list = dois.to_list()
print(f"Detected {len(doi_list)} DOIs to search.")

# --- API REQUEST ---
token = "pHazHxvHjPVPAcotvj7DIijROZXUjG5vXa2OaCQO"
url = "https://api.adsabs.harvard.edu/v1/search/query?"

data = []

for doi in doi_list:
    query = f"doi:{doi}"
    params = {"q": query, "fl": "doi,bibcode", "rows": 1}
    headers = {'Authorization': 'Bearer ' + token}
    response = requests.get(url, params=params, headers=headers)
    
    # Check for HTTP errors
    try:
        response.raise_for_status()
    except requests.HTTPError as e:
        print(f"HTTP error: {e}")
        continue  # Skip to the next iteration

    # Check if response content is empty
    if response.content:
        try:
            from_solr = response.json()
            if from_solr.get('response'):
                num_docs = from_solr['response'].get('numFound', 0)
                if num_docs > 0:
                    for doc in from_solr['response']['docs']:
                        result = (doc['bibcode'], doc['doi'][0])
                        print(result)
                        data.append(result)
        except json.JSONDecodeError as json_err:
            print(f"JSON decoding error: {json_err}")
            continue  # Skip to the next iteration
    else:
        print("Empty response")
        continue  # Skip to the next iteration

# Create a DataFrame from the API results
dois_matched = pd.DataFrame(data, columns=['preprint_bib', 'preprint_doi'])

# Merge the results with the original DataFrame
# Use a left join to bring in the bibcodes for rows that match on 'preprint_doi'
merged = df.merge(dois_matched, on='preprint_doi', how='left', suffixes=('', '_new'))

# Update 'preprint_bib' only where the current value is '...................'
# and there's a new value from the API
merged.loc[merged['preprint_bib'] == '...................', 'preprint_bib'] = merged['preprint_bib_new']

# Drop the temporary '_new' column created by the merge
merged = merged.drop(columns=['preprint_bib_new'])

# Export merged data
merged.to_excel(excel_file, index=False)
print(f"Merged {len(dois_matched)} matching bibcodes")


## Harvest from CrossRef

In [ ]:
cr = Crossref()
doi_prefix = '10.31223'
res = cr.prefixes(ids = doi_prefix, works = True, cursor = "*", limit = 200)

records = []
for entry in res:
    for item in entry['message']['items']:

        def format_authors(authors_data):
            formatted_authors = []
            for author in authors_data:
                last_name = author.get('family', '')
                first_name = author.get('given', '')
                if last_name and first_name:
                    formatted_authors.append(f"{last_name}, {first_name}")
                elif last_name and not first_name:
                    formatted_authors.append(last_name)
            return "; ".join(formatted_authors)
        
        def format_affs(authors_data):
            formatted_affs = []
            for author in authors_data:
                aff = author.get('affiliation', '')
                orcid = author.get('ORCID', '').lstrip("http://orcid.org/")
                formatted_aff = ""
                if aff:
                    formatted_aff = f'{aff}'
                if orcid:
                    formatted_aff += f'<ID system=\"ORCID\">{orcid}</ID>'
                formatted_affs.append(formatted_aff)
            return "; ".join(formatted_affs)

        authors_data = item.get('author', [])
        authors = format_authors(authors_data)
        affiliations = format_affs(authors_data)
        title = item.get('title', '')[0]
        group_title = item.get('group-title', '')
        abstract = item.get('abstract', '').replace("<jats:p>", "").replace("</jats:p>", "")
        preprint_doi = item.get('DOI', '')
        url = item.get('resource', {}).get('primary', {}).get('URL', '')
        
        links = ""
        if preprint_doi:
            links += f"DOI: {preprint_doi}"
        if url:
            links += f"; ELECTR: {url}"
        links = links.lstrip("; ").rstrip("/")
        
        pubdate = ""
        if "published" in item and "date-parts" in item["published"]:
            date_parts = item["published"]["date-parts"]
            if date_parts:
                year, month, day = date_parts[0]
                pubdate = f"{year}/{month:02d}/{day:02d}"
            
        article_doi = ""
        if "relation" in item and "is-preprint-of" in item["relation"]:
            is_preprint_of = item["relation"]["is-preprint-of"]
            if isinstance(is_preprint_of, list) and len(is_preprint_of) > 0:
                article_doi = (is_preprint_of[0]["id"]).lstrip("https://doi.org/").lstrip("x.doi.org/")
    
        wd_abs = abstract == "Abstract removed when submission was withdrawn."
        if title != "":
            r = {
                "authors": "; ".join(authors.split("; ")),
                "affiliations": "; ".join(affiliations.split("; ")),
                "pubdate":pubdate,
                "title": title,
                "properties": links,
                "abstract": abstract,
                "keywords": group_title,
                "preprint_doi": preprint_doi,
                "article_doi": article_doi,
                "source": "CrossRef",
                "preprint_bib": "...................",
                "processed": "withdrawn" if wd_abs else str(today_date)
            }
            records.append(r)

## Curation: Add new records and metadata changes to data file

In [ ]:
# Create a set of preprint_dois already harvested
master_file = pd.read_excel(excel_file)
master_data = pd.DataFrame(master_file)
exclusions = set(master_file['preprint_doi'].tolist())

# Create a list of the new records based on new preprint_dois
records_to_add = []
metadata_updates = []

def clean_value(value):
    """Helper function to clean NaN values by converting them to empty strings."""
    if pd.isna(value) or value is None:
        return ""
    return str(value)

for r in records:
    
    eaarx_doi = r['preprint_doi']

    # If the DOI does not yet exist in the master file, append the whole record for the ingest list
    if eaarx_doi not in exclusions:
        records_to_add.append(r)

    # If the DOI exists, check each metadata field for changes from the master file
    elif eaarx_doi in exclusions:
        # Get the corresponding row in the master file for comparison
        master_row = master_data[master_data['preprint_doi'] == eaarx_doi].iloc[0]

        updated_fields = {}
        
        if clean_value(master_row['preprint_bib']) != "withdrawn" and clean_value(master_row['preprint_bib']) != "bad":
            # Compare each relevant field, cleaning NaN values
            if clean_value(r['authors']) != clean_value(master_row['authors']):
                updated_fields['authors'] = {'old': clean_value(master_row['authors']), 'new': clean_value(r['authors'])}
            if clean_value(r['affiliations']) != clean_value(master_row['affiliations']):
                updated_fields['affiliations'] = {'old': clean_value(master_row['affiliations']), 'new': clean_value(r['affiliations'])}
            if clean_value(r['title']) != clean_value(master_row['title']):
                updated_fields['title'] = {'old': clean_value(master_row['title']), 'new': clean_value(r['title'])}
            if clean_value(r['pubdate']) != clean_value(master_row['pubdate']):
                updated_fields['pubdate'] = {'old': clean_value(master_row['pubdate']), 'new': clean_value(r['pubdate'])}
            if clean_value(r['abstract']) != clean_value(master_row['abstract']):
                updated_fields['abstract'] = {'old': clean_value(master_row['abstract']), 'new': clean_value(r['abstract'])}
            if clean_value(r['article_doi']) != clean_value(master_row['article_doi']):
                updated_fields['article_doi'] = {'old': clean_value(master_row['article_doi']), 'new': clean_value(r['article_doi'])}
    
            # If there are any updates, append to metadata_updates
            if updated_fields:
                r = {
                    'preprint_doi': eaarx_doi,
                    'updates': updated_fields
                }
                print(r, '\n')
                metadata_updates.append(r)

# Now metadata_updates will contain all the changes made to existing records
print(f"Harvester detected {len(records_to_add)} new records.")
print(f"Metadata changes detected for {len(metadata_updates)} records.")

In [ ]:
# Update the master_data DataFrame with the metadata changes
for update in metadata_updates:
    preprint_doi = update['preprint_doi']
    
    # Extract the 'new' values from the updates
    new_values = {key: value['new'] for key, value in update['updates'].items()}
    
    # Locate the row in the master_data where the preprint_doi matches
    master_data.loc[master_data['preprint_doi'] == preprint_doi, list(new_values.keys())] = list(new_values.values())

# Merge the new records with the original data and remove duplicates based on preprint_doi
new_records_df = pd.DataFrame(records_to_add)
merged_df = pd.concat([master_data, new_records_df], ignore_index=True)
merged_df = merged_df.drop_duplicates(subset=["preprint_doi"])

# Write the merged DataFrame back to the master excel file
merged_df.to_excel(excel_file, index=False)
print(f"Updated existing metadata and added {len(records_to_add)} new records to {excel_file}")

## Curation: Generate tagged format records

In [ ]:
import re
from pyingest.serializers.classic import Tagged

### Function to Process Records and Generate Tagged Format ###
def process_records(records, output_file):
    # Pyingest Serializer - Transform records into tagged format
    with open(output_file, 'a') as outputfp:
        for record in records:
            try:
                serializer = Tagged()
                serializer.write(record, outputfp)
            except Exception as e:
                print(f"Serializer failed for record: {record}, Error: {e}")
    
    # Post-processing: Perform &amp; to & replacement
    with open(output_file, 'r') as file:
        data = file.read()
        data = re.sub(r'&amp;', '&', data)
    
    # Write the modified content back to the file
    with open(output_file, 'w') as file:
        file.write(data)

### TAG FILE FOR NEW RECORDS HARVEST ###
# Prepare the records for tagging
add_records = pd.DataFrame(records_to_add)
new_records = []
for index, row in add_records.iterrows():
    if row["abstract"] != "Abstract removed when submission was withdrawn.":  # Check abstract condition
        r = {
            "authors": [] if pd.isna(row["authors"]) else row["authors"].split("; "),
            "affiliations": [] if pd.isna(row["affiliations"]) else row["affiliations"].split("; "),
            "pubdate": "" if pd.isna(row["pubdate"]) else row["pubdate"],
            "title": "" if pd.isna(row["title"]) else row["title"],
            "properties": [] if pd.isna(row["properties"]) else row["properties"].split("; "),
            "abstract": "" if pd.isna(row["abstract"]) else row["abstract"],
            "keywords": "" if pd.isna(row["keywords"]) else row["keywords"],
            "comments": "" if pd.isna(row["article_doi"]) else f'Published article doi: {row["article_doi"]}',
            "source": "CrossRef"
        }
        new_records.append(r)

# Process and generate the tagged file for new records
process_records(new_records, harvest_tagged_output)
print(f"Saved {len(new_records)} new records to {harvest_tagged_output}")

# Prepare the metadata updates for tagging
updated_records = []
for update in metadata_updates:
    preprint_doi = update['preprint_doi']
    
    # Find the corresponding entry in master_data to get preprint_bib and old article doi
    master_row = master_data.loc[master_data['preprint_doi'] == preprint_doi]
    preprint_bib = master_row['preprint_bib'].values
    old_article_doi = master_row['article_doi'].values[0] if not master_row.empty else None

    # Extract the new article DOI and preprint DOI
    new_article_doi = update["updates"].get("article_doi", {}).get("new", "").lstrip("x.doi.org/")
    new_preprint_doi = update["updates"].get("preprint_doi", {}).get("new", "").lstrip("x.doi.org/")
    
    # Skip updates that only change pubdate or where new preprint DOI matches the old
    if (len(update["updates"]) == 1 and 'pubdate' in update["updates"]) or \
       (old_article_doi and str(old_article_doi).lstrip("x.doi.org/") == str(new_preprint_doi).lstrip("x.doi.org/")):
        continue  # Skip this update

    # Create the comment conditionally
    comments = f'Published article doi: {new_article_doi}' if new_article_doi else ""

    r = {
        "authors": update["updates"].get("authors", {}).get("new", ""),
        "affiliations": update["updates"].get("affiliations", {}).get("new", ""),
        "title": update["updates"].get("title", {}).get("new", ""),
        "properties": update["updates"].get("properties", {}).get("new", ""),
        "abstract": update["updates"].get("abstract", {}).get("new", ""),
        "keywords": update["updates"].get("keywords", {}).get("new", ""),
        "comments": comments,
        "bibcode": preprint_bib[0] if len(preprint_bib) > 0 else "",  # Get preprint_bib or default to empty string
    }
    
    updated_records.append(r)

# Process and generate the tagged file for metadata updates
process_records(updated_records, metadata_tagged_output)
print(f"Saved {len(updated_records)} updated records to {metadata_tagged_output}")

## (*Not in use*) Resolve published article DOIs for matching bibcodes

In [ ]:
# import pandas as pd
# import requests
# import json

# # Import new excel sheet
# excel_file = "eartharxiv_data.xlsx"
# path = "/Users/sao/Documents/Python-Projects/EarthArXiv/"
# df = pd.read_excel(path + excel_file)

# # Filter the DataFrame based on conditions for 'article_doi' and 'article_bib'
# filtered_df = df[(~df['article_doi'].isnull()) & (df['article_bib'].isnull())]

# # Isolate the DOIs and drop all the rows that have no DOIs (drop null values)
# dois = filtered_df['article_doi'].dropna()

# # Convert it from a DataFrame to a list
# doi_list = dois.to_list()
# print("Filtered list has", len(doi_list), "DOIs to search.")

# # --- API REQUEST --- 
# token = "pHazHxvHjPVPAcotvj7DIijROZXUjG5vXa2OaCQO"
# url = "https://api.adsabs.harvard.edu/v1/search/query?"

# data = []

# for doi in doi_list:
#     query = f"doi:{doi}"
    
#     params = {"q": query, "fl": "doi,bibcode", "rows": 1}
#     headers = {'Authorization': 'Bearer ' + token}
#     response = requests.get(url, params=params, headers=headers)
    
#     # Check for HTTP errors
#     try:
#         response.raise_for_status()
#     except requests.HTTPError as e:
#         print(f"HTTP error: {e}")
#         continue  # Skip to the next iteration

#     # Check if response content is empty
#     if response.content:
#         try:
#             from_solr = response.json()
#             if from_solr.get('response'):
#                 num_docs = from_solr['response'].get('numFound', 0)
#                 if num_docs > 0:
#                     for doc in from_solr['response']['docs']:
#                         result = (doc['bibcode'], doc['doi'][0])
#                         print(result)
#                         data.append(result)
#         except json.JSONDecodeError as json_err:
#             print(f"JSON decoding error: {json_err}")
#             continue  # Skip to the next iteration
#     else:
#         print("Empty response")
#         continue  # Skip to the next iteration

# dois_matched = pd.DataFrame(data, columns=['article_bib', 'article_doi'])
# merged = df.merge(dois_matched, left_on='article_doi', right_on='article_doi', how='left')

# # Export merged data
# merged.to_excel(path + excel_file, index=False)
# print(f"Merged {len(dois_matched)} bibcodes")